## CESM2 - LARGE ENSEMBLE (LENS2)

- The purpose of this notebook is to calculate the time series of the annual cycle and the anomaly of volume transport and heat content. 

### Imports

In [ ]:
# modules I am using in this example
import xarray as xr
import xgcm
from xgcm import Grid
import pop_tools
from dask.distributed import Client, wait
from ncar_jobqueue import NCARCluster
import dask
import intake
import intake_esm
import dask
import numpy as np 
import matplotlib.pyplot as plt
import warnings, getpass, os

### Dask

In [ ]:
mem_per_worker = 50 # memory per worker in GB 
num_workers = 40 # number of workers
cluster = NCARCluster(cores=1, processes=1, memory=f'{mem_per_worker} GB',resource_spec=f'select=1:ncpus=1:mem={mem_per_worker}GB', walltime='1:00:00')
cluster.scale(num_workers)
client = Client(cluster)
print(client)
client

### Load data

In [ ]:
catalog = intake.open_esm_datastore(
    '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
)
cat_subset = catalog.search(component='ocn',variable=['VVEL'],frequency='month_1')
# Load catalog entries for subset into a dictionary of xarray datasets
dset_dict_raw  = cat_subset.to_dataset_dict(zarr_kwargs={'consolidated': True},cdf_kwargs={'chunks': {'time': 12,'z_t': 1}}, storage_options={'anon': True})
print(f'\nDataset dictionary keys:\n {dset_dict_raw.keys()}')

### Concatenation of variables

In [ ]:
ff=('cmip6','smbb')               # Forcings
fb=(['VVEL']) # Variable

ds_dict = dict()
for var in fb:
    # 1- combine historical and ssp370 (concatenate in time)
    ds_dict_tmp = dict()
    for scenario in ff:
        ds_dict_tmp[scenario] = xr.combine_nested([dset_dict_raw[f'ocn.historical.pop.h.{scenario}.{var}'], dset_dict_raw[f'ocn.ssp370.pop.h.{scenario}.{var}']],concat_dim=['time'])
        
        # 2- combine cmip6 and smbb (concatenate in member_id)
    ds_dict[var] = xr.combine_nested([ds_dict_tmp['cmip6'], ds_dict_tmp['smbb']], concat_dim=['member_id'])
    del ds_dict_tmp

### Import POP grid

In [ ]:
pop_grid = pop_tools.get_grid('POP_gx1v7')
ds_dict['TLONG'] = pop_grid.TLONG; ds_dict['TLAT'] = pop_grid.TLAT
ds_dict['ULONG'] = pop_grid.ULONG; ds_dict['ULAT'] = pop_grid.ULAT
del pop_grid

### Cut and center the variable in the South Atlantic

In [ ]:
%%time
# Cutting out and centering the variables in the South Atlantic
dask.config.set({"array.slicing.split_large_chunks": True})
ilon1, flon1, ilon2, flon2 = 307, 320, 0, 54 # longitude (initial (i), final (f)) 
ilan = 0 # northernmost latitude
ilas = -34 # southernmost latitude

fb=(['VVEL'])

for var in fb:
    ds_dict[f'{var}']=xr.combine_nested([[
        ds_dict[f'{var}'].where((ds_dict[f'{var}'].TLAT >= ilas) & (ds_dict[f'{var}'].TLAT <= ilan), drop=True).isel(
            nlon = slice(ilon1,flon1)),
        ds_dict[f'{var}'].where((ds_dict[f'{var}'].TLAT >= ilas) & (ds_dict[f'{var}'].TLAT <= ilan), drop=True).isel(
            nlon = slice(ilon2,flon2))]],
        concat_dim=['nlat','nlon'])   
    ds_dict[f'{var}'].coords['nlon'] = (ds_dict[f'{var}'].coords['nlon'] + 180) % 360 - 180 
    ds_dict[f'{var}'] = ds_dict[f'{var}'].sortby(ds_dict[f'{var}'].nlon)
del ilan, ilas, ilon1, flon1, ilon2, flon2 

### Mask the continent 

#### Let's split heat transport into velocity $(\rm{V})$ and temperature $(\rm{T})$ components as follows:
#### Equation (1): $$\rm{VT} = (\rm{\bar{V}+V^{'})(\bar{T}+T^{'})},$$
#### Equation (2): $$\rm{VT} = \rm{\bar{V}\bar{T}+\bar{V}T^{'}+V^{'}\bar{T}+V^{'}T^{'}},$$
##### where $(\bar{})$ represents the climatology annual mean and $(^{'})$  represents the anomaly regarding the annual mean. However in place of temperature we will use the Heat Storage (HS) previously calculated (see Notebook_lens2_South_Atlantic_heat_balance.ipynb) and instead of velocity we will use the volume transport calculated as follows:

#### Vertical Integration
#### Equation (3): $$\rm{VINT_Z = \int_{z_2}^{z_1}VVEL~dz},$$
##### where:
##### * $\rm{VVEL}$ ($\rm{m~s^{-1}}$) is the meridional velocity,
##### * $\rm{VINT_Z}$ ($\rm{m^{2}~s^{-1}}$) is the vertically integrated meridional velocity,
##### * $\rm{z_1}$ and $\rm{z_2}$ are the depths limit on the integral. We consider the integral of the velocity from the surface to the bottom, and
##### * $\rm{dz}$ ($\rm{m}$) is the spacing of each velocity cell. 
#### Zonal integration
#### Equation (4): $$\rm{VINT_{ZX} = \int_{x_2}^{x_1}VINT_Z~DXT},$$
##### where:
##### * $\rm{VINT_{ZX}}$ ($\rm{m^{3}~s^{-1}}$) is the vertically integrated meridional velocity and integrated zonally, i.e., the volume transport,
##### * $\rm{x_1}$ and $\rm{x_2}$ are the zonal limit on the integral. We consider the integral of the velocity from from the west side to the east side of the basin, and
##### * $\rm{DXT}$ ($\rm{m}$) is the x-spacing centered at T points.
#### Differentiation
#### Equation (5): $$\rm{Vol = VINT^{ilan}_{ZX}-VINT^{ilas}_{ZX}},$$
##### where $\rm{Vol}$ ($\rm{m^{3}~s^{-1}}$) is the difference in volume transport ($\rm{VINT_{ZX}}$) between the northernmost latitude ($\rm{ilan}$) and the southernmost latitude ($\rm{ilas}$).

#### Volume Transport

In [ ]:
dx=ds_dict['VVEL']['DXT'].isel(time=0,member_id=0)*0.01 # (0.01 to convert cm into m)
dz=ds_dict['VVEL']['dz'].isel(member_id=0,time=0,nlon=0,nlat=0)*0.01 # (0.01 to convert cm into m)

In [ ]:
%%time
warnings.simplefilter("ignore")
Vol_array = list() # Build a list
# Volume Transport
for member_id in range(len(ds_dict['VVEL']['VVEL'].coords['member_id'])-99):  
    VINTZ=(ds_dict['VVEL']['VVEL'].isel(member_id=member_id)*dz).sum(dim=['z_t']) # m2/s (0.01 to convert cm into m)
    VINTZX=(VINTZ*dx).sum(dim=['nlon']) # m3/s 
    Vol=(VINTZX.isel(nlat=-1)-VINTZX.isel(nlat=0)).compute() # m3/s
    del VINTZX, VINTZ
    Vol_array.append(Vol) 
    print(f'Done with member: {member_id}')
Vol_array_merged = xr.concat(Vol_array, dim='member_id', compat='override', join='override', coords='minimal') # concat the members      

In [ ]:
ds_out_var = xr.merge([Vol_array_merged.rename('Vol')]) # Volume Transport
ds_out_var.attrs['description'] = 'Volume transport in the South Atlantic'
ds_out_var.attrs['units'] = 'm3/s'
ds_out_var.attrs['author'] = 'Mauricio Rocha'
ds_out_var.attrs['email'] = 'mauricio.rocha@usp.br'
# create a directory on scratch to save the output
path = '/glade/scratch/mauricio/Data/LENS2/HEAT_BALANCE/'.format(getpass.getuser())
os.system('mkdir -p '+path)
ds_out_var.to_netcdf(path+'volume_transport.nc')

In [ ]:
climatology_vol=xr.DataArray(np.tile(ds_out_var.Vol.groupby('time.month').mean('time').values,(1,251)), 
                         coords=ds_out_var.Vol.coords, 
                         dims=ds_out_var.Vol.dims, 
                         attrs=ds_out_var.Vol.attrs)
anomalies_vol= ds_out_var.Vol.groupby('time.month') - ds_out_var.Vol.groupby('time.month').mean('time')

In [ ]:
ds = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/HEAT_BALANCE/heat_balance_components.nc')
ds=((ds/1026)/3996)*1e+15
climatology_HS=xr.DataArray(np.tile(ds.HS_TEND_TEMP.groupby('time.month').mean('time').values,(1,251)), 
                         coords=ds.HS_TEND_TEMP.coords, 
                         dims=ds.HS_TEND_TEMP.dims, 
                         attrs=ds.HS_TEND_TEMP.attrs)
anomalies_HS= ds.HS_TEND_TEMP.groupby('time.month') - ds.HS_TEND_TEMP.groupby('time.month').mean('time')

In [ ]:
plt.figure(figsize=(10, 5))
(climatology_HS.resample(time='1Y',closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).isel(
    member_id=0)*climatology_vol.resample(time='1Y',closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).isel(
    member_id=0)).plot.line(x='time',color='blue',label=r'$\rm{\bar{V}\bar{T}}$')


(climatology_HS.resample(time='1Y',closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).isel(
    member_id=0)*anomalies_vol.resample(time='1Y',closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).isel(
    member_id=0)).plot.line(x='time',color='red',label=r'$\rm{\bar{T}V{^\prime}}$')


(anomalies_HS.resample(time='1Y',closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).isel(
    member_id=0)*climatology_vol.resample(time='1Y',closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).isel(
    member_id=0)).plot.line(x='time',color='green',label=r'$\rm{\bar{V}T{^\prime}}$')


(anomalies_HS.resample(time='1Y',closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).isel(
    member_id=0)*anomalies_vol.resample(time='1Y',closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).isel(
    member_id=0)).plot.line(x='time',color='black',label=r'$\rm{V{^\prime}T{^\prime}}$')
plt.grid(color='gray', linestyle='-', linewidth=0.7)
plt.legend()
plt.title(None)
plt.ylabel(r'$\rm{{^{o}C}}m^{6}s^{-2}$')
plt.xlabel('Time [Years]')
plt.savefig('heat_transport_decomposition.png',dpi=300,bbox_inches='tight')
plt.show()